In [25]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

In [26]:
df=pd.read_csv("C:\\Users\\saipr\\Crop_Recommendation\\data\\Crop_recommendation.csv")

In [27]:
df 

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice
...,...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507,coffee
2196,99,15,27,27.417112,56.636362,6.086922,127.924610,coffee
2197,118,33,30,24.131797,67.225123,6.362608,173.322839,coffee
2198,117,32,34,26.272418,52.127394,6.758793,127.175293,coffee


In [28]:
df['label'].value_counts()

label
rice           100
maize          100
jute           100
cotton         100
coconut        100
papaya         100
orange         100
apple          100
muskmelon      100
watermelon     100
grapes         100
mango          100
banana         100
pomegranate    100
lentil         100
blackgram      100
mungbean       100
mothbeans      100
pigeonpeas     100
kidneybeans    100
chickpea       100
coffee         100
Name: count, dtype: int64

## Splitting the Data

In [29]:
X=df.drop('label',axis=1)
y=df['label']

In [30]:
from sklearn.model_selection import train_test_split

# Assuming X is your feature set and y is the corresponding label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [67]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

mlflow.set_tracking_uri("http://127.0.0.1:5000")

mlflow.set_experiment("RandomForest_GridSearch")

with mlflow.start_run():
    rf = RandomForestClassifier(random_state=42)

    params = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30], 
        'min_samples_split': [2, 5, 10],  
        'min_samples_leaf': [1, 2, 4],  
        'criterion': ['gini', 'entropy']  
    }

    grid_search_rf = GridSearchCV(estimator=rf, param_grid=params, 
                               cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

    grid_search_rf.fit(X_train, y_train)

    best_params = grid_search_rf.best_params_
    best_score = grid_search_rf.best_score_

    mlflow.log_params(best_params)
    mlflow.log_metric("best_accuracy", best_score)

    mlflow.sklearn.log_model(grid_search_rf.best_estimator_, "best_random_forest_model")

    print("Best Parameters:", best_params)
    print("Best Accuracy:", best_score)

    mlflow.end_run()


2025/02/16 01:16:03 INFO mlflow.tracking.fluent: Experiment with name 'RandomForest_GridSearch' does not exist. Creating a new experiment.


Fitting 5 folds for each of 216 candidates, totalling 1080 fits


2025/02/16 01:18:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Best Parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Best Accuracy: 0.9960227272727271
🏃 View run illustrious-cub-960 at: http://127.0.0.1:5000/#/experiments/809510633914373352/runs/7be5a439659d4274a47adfc717813c77
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/809510633914373352


In [68]:
model_rfc=grid_search_rf.best_estimator_

In [69]:
grid_search_rf.best_params_

{'criterion': 'gini',
 'max_depth': 10,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 100}

In [70]:
grid_search_rf.best_score_

0.9960227272727271

In [71]:
from sklearn.metrics import accuracy_score,confusion_matrix
y_pred=model_rfc.predict(X_test)
accuracy_score(y_test,y_pred)

0.9954545454545455

In [72]:
print(confusion_matrix(y_test,y_pred))

[[20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 19  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0

## Trying Out Support Vector Machines Algorithm

In [73]:
import mlflow
import mlflow.sklearn
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

mlflow.set_tracking_uri("http://127.0.0.1:5000")

mlflow.set_experiment("SVC_GridSearch")

with mlflow.start_run():
    svc = SVC()

    params = {
        'C': [0.1, 1, 10, 100],  
        'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
    }

    grid_search_svc = GridSearchCV(estimator=svc, param_grid=params, 
                                   cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

    grid_search_svc.fit(X_train, y_train)

    best_params = grid_search_svc.best_params_
    best_score = grid_search_svc.best_score_

    mlflow.log_params(best_params)
    mlflow.log_metric("best_accuracy", best_score)

    mlflow.sklearn.log_model(grid_search_svc.best_estimator_, "best_svc_model")

    print("Best Parameters:", best_params)
    print("Best Accuracy:", best_score)

    mlflow.end_run()


2025/02/16 01:21:04 INFO mlflow.tracking.fluent: Experiment with name 'SVC_GridSearch' does not exist. Creating a new experiment.


Fitting 5 folds for each of 16 candidates, totalling 80 fits


2025/02/16 01:21:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Best Parameters: {'C': 0.1, 'kernel': 'linear'}
Best Accuracy: 0.9857954545454545
🏃 View run agreeable-goose-248 at: http://127.0.0.1:5000/#/experiments/308727505154579858/runs/b513565508424ee1883922873cb0fa35
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/308727505154579858


In [74]:
model_svc=grid_search_svc.best_estimator_

In [75]:
y_pred1=model_svc.predict(X_test)

In [76]:
accuracy_score(y_test,y_pred1)

0.9931818181818182

In [77]:
print(confusion_matrix(y_test,y_pred1))

[[20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0 19  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0 19  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0

## Gradient Boosting

In [78]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

mlflow.set_tracking_uri("http://127.0.0.1:5000")

mlflow.set_experiment("GradientBoosting_GridSearch")

with mlflow.start_run():
    gb = GradientBoostingClassifier(random_state=42)

    params = {
        'n_estimators': [50, 100, 150],  
        'learning_rate': [0.01, 0.05, 0.1],  
    }

    grid_search_gb = GridSearchCV(estimator=gb, param_grid=params, 
                                  cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

    grid_search_gb.fit(X_train, y_train)

    best_params = grid_search_gb.best_params_
    best_score = grid_search_gb.best_score_

    mlflow.log_params(best_params)
    mlflow.log_metric("best_accuracy", best_score)

    mlflow.sklearn.log_model(grid_search_gb.best_estimator_, "best_gradient_boosting_model")

    print("Best Parameters:", best_params)
    print("Best Accuracy:", best_score)

    mlflow.end_run()


2025/02/16 01:24:17 INFO mlflow.tracking.fluent: Experiment with name 'GradientBoosting_GridSearch' does not exist. Creating a new experiment.


Fitting 5 folds for each of 9 candidates, totalling 45 fits


2025/02/16 01:25:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Best Parameters: {'learning_rate': 0.1, 'n_estimators': 100}
Best Accuracy: 0.9875
🏃 View run rare-auk-421 at: http://127.0.0.1:5000/#/experiments/148959594547896690/runs/417ba78c49f845bd9ff43a5f9a381fb2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/148959594547896690


In [79]:
model_gb=grid_search_gb.best_estimator_

In [80]:
y_pred2=model_gb.predict(X_test)

In [81]:
accuracy_score(y_test,y_pred2)

0.9886363636363636

In [82]:
print(confusion_matrix(y_test,y_pred2))

[[20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 19  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 19  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0